<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [3]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=np.mean(train_x), variance=np.var(train_x)),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [4]:
def objective(params):
    # MLflow will track the parameters and results for each run
    with mlflow.start_run():
        result = train_model(
            params,
            epochs=3,
            train_x=train_x,
            train_y=train_y,
            valid_x=valid_x,
            valid_y=valid_y,
            test_x=test_x,
            test_y=test_y,
        )
        mlflow.log_params(params)
        mlflow.log_metric('eval_rmse', result['loss'])
        mlflow.tensorflow.log_model(result["model"], "model", signature=signature)
        return result

In [5]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [6]:
# Set the MLflow experiment name
mlflow.set_experiment("/wine-quality")

# Conduct the hyperparameter search using Hyperopt
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=8,
    trials=trials,
)

# Fetch the details of the best run
best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

# Log the best parameters and loss outside of the 'objective' function
with mlflow.start_run():
    mlflow.log_params(best)
    mlflow.log_metric("best_loss", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "best_model", signature=signature)

# Print out the best parameters and corresponding loss
print(f"Best parameters: {best}")
print(f"Best eval rmse: {best_run['loss']}")

2024/02/02 00:08:48 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


  0%|                                     | 0/8 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                                                       

46/46 [==============================] - 0s 3ms/step - loss: 6.2528 - root_mean_squared_error: 2.5006 - val_loss: 0.8726 - val_root_mean_squared_error: 0.9341

Epoch 2/3                                                                       

46/46 [==============================] - 0s 962us/step - loss: 0.7999 - root_mean_squared_error: 0.8944 - val_loss: 0.7457 - val_root_mean_squared_error: 0.8636

Epoch 3/3                                                                       

46/46 [==============================] - 0s 930us/step - loss: 0.7732 - root_mean_squared_error: 0.8793 - val_loss: 0.7486 - val_root_mean_squared_error: 0.8652

12/12 [==============================] - 0s 831us/step - loss: 0.7486 - root_mean_squared_error: 0.8652

  0%|                                     | 0/8 [00:00<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/t

INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/tmpjqwkecr4/model/data/model/assets



INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/tmp_2xm1irw/model/data/model/assets


...

INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/tmpi9gwc7kg/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/tmpi9gwc7kg/model/data/model/assets



100%|███████████| 8/8 [01:28<00:00, 11.11s/trial, best loss: 0.8652371764183044]
INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/tmpx2i40v9r/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/43/zgw1tdw96479k4wmj51vd58h0000gn/T/tmpx2i40v9r/model/data/model/assets



Best parameters: {'lr': 0.05544611489404935, 'momentum': 0.010745082016461627}
Best eval rmse: 0.8652371764183044


In [7]:
!mlflow ui

[2024-02-02 00:10:46 -0500] [21245] [INFO] Starting gunicorn 21.2.0
[2024-02-02 00:10:46 -0500] [21245] [INFO] Listening at: http://127.0.0.1:5000 (21245)
[2024-02-02 00:10:46 -0500] [21245] [INFO] Using worker: sync
[2024-02-02 00:10:46 -0500] [21246] [INFO] Booting worker with pid: 21246
[2024-02-02 00:10:46 -0500] [21247] [INFO] Booting worker with pid: 21247
[2024-02-02 00:10:46 -0500] [21248] [INFO] Booting worker with pid: 21248
[2024-02-02 00:10:46 -0500] [21249] [INFO] Booting worker with pid: 21249
^C
[2024-02-02 00:14:07 -0500] [21245] [INFO] Handling signal: int
[2024-02-02 00:14:07 -0500] [21249] [INFO] Worker exiting (pid: 21249)
[2024-02-02 00:14:07 -0500] [21246] [INFO] Worker exiting (pid: 21246)
[2024-02-02 00:14:07 -0500] [21248] [INFO] Worker exiting (pid: 21248)
[2024-02-02 00:14:07 -0500] [21247] [INFO] Worker exiting (pid: 21247)
